# Prolog Hello World

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/bacalhau-project/examples/blob/main/workload-onboarding/Prolog-Hello-World/index.ipynb)
[![Open In Binder](https://mybinder.org/badge.svg)](https://mybinder.org/v2/gh/bacalhau-project/examples/HEAD?labpath=workload-onboarding/Prolog-Hello-World/index.ipynb)

# Running Prolog on bacalhau
## Introduction
Prolog is intended primarily as a declarative programming language: the program logic is expressed in terms of relations, represented as facts and rules. A computation is initiated by running a query over these relations.
Prolog is well-suited for specific tasks that benefit from rule-based logical queries such as searching databases, voice control systems, and filling templates.
In this example we will run a hello world prolog script on bacalhau



## Running Locally​


Install swipl 


In [ ]:
%%bash
sudo add-apt-repository ppa:swi-prolog/stable
sudo apt-get update
sudo apt-get install swi-prolog

 Comprehensive Prolog implementation with extensive libraries and development tools.   Primarily targetted at teaching, RDF processing and web-related tasks, such as creating web services or analysing web content.

Official PPAs for SWI-Prolog. See https://www.swi-prolog.org for further information.
 More info: https://launchpad.net/~swi-prolog/+archive/ubuntu/stable
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-up

Create a file called helloworld.pl 
This prints ‘Hello World’ to the stdout


In [ ]:
%%writefile helloworld.pl
hello_world :- write('Hello World'), nl, 
               halt.

Overwriting helloworld.pl


Running the script


In [ ]:
%%bash
swipl -q -s helloworld.pl -g hello_world

Hello World


After the script has run successfully locally we can now run it on bacalhau

For that we upload our script to IPFS since the script is only present locally
ipfs add helloworld.pl

Using IPFS cli


In [ ]:
!wget https://dist.ipfs.io/go-ipfs/v0.4.2/go-ipfs_v0.4.2_linux-amd64.tar.gz
!tar xvfz go-ipfs_v0.4.2_linux-amd64.tar.gz
!mv go-ipfs/ipfs /usr/local/bin/ipfs
!ipfs init
!ipfs cat /ipfs/QmYwAPJzv5CZsnA625s3Xf2nemtYgPpHdWEz79ojWnPbdG/readme
!ipfs config Addresses.Gateway /ip4/127.0.0.1/tcp/8082
!ipfs config Addresses.API /ip4/127.0.0.1/tcp/5002
!nohup ipfs daemon > startup.log &

--2022-11-12 08:44:36--  https://dist.ipfs.io/go-ipfs/v0.4.2/go-ipfs_v0.4.2_linux-amd64.tar.gz
Resolving dist.ipfs.io (dist.ipfs.io)... 209.94.78.1, 2602:fea2:3::1
Connecting to dist.ipfs.io (dist.ipfs.io)|209.94.78.1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7642422 (7.3M) [application/gzip]
Saving to: ‘go-ipfs_v0.4.2_linux-amd64.tar.gz.4’

go-ipfs_v0.4.2_linu 100%[===================>]   7.29M  --.-KB/s    in 0.04s   

2022-11-12 08:44:36 (206 MB/s) - ‘go-ipfs_v0.4.2_linux-amd64.tar.gz.4’ saved [7642422/7642422]

go-ipfs/build-log
go-ipfs/install.sh
go-ipfs/ipfs
go-ipfs/LICENSE
go-ipfs/README.md
Error: ipfs daemon is running. please stop it to run this command
Use 'ipfs init --help' for information about this command
Hello and Welcome to IPFS!

██╗██████╗ ███████╗███████╗
██║██╔══██╗██╔════╝██╔════╝
██║██████╔╝█████╗  ███████╗
██║██╔═══╝ ██╔══╝  ╚════██║
██║██║     ██║     ███████║
╚═╝╚═╝     ╚═╝     ╚══════╝

If you're seeing this, you have successful

In [ ]:
!ipfs add helloworld.pl

added QmYq9ipYf3vsj7iLv5C67BXZcpLHxZbvFAJbtj7aKN5qii helloworld.pl


Using NFTup

image.png

Running on bacalhau

We will use the official [swipl docker image](https://hub.docker.com/_/swipl/)


In [ ]:
!curl -sL https://get.bacalhau.org/install.sh | bash

Your system is linux_amd64
No BACALHAU detected. Installing fresh BACALHAU CLI...
Getting the latest BACALHAU CLI...
Installing v0.3.11 BACALHAU CLI...
Verified OK
Extracting tarball ...
NOT verifying Bin
bacalhau installed into /usr/local/bin successfully.
Client Version: v0.3.11
Server Version: v0.3.11


Command


In [ ]:
%%bash --out job_id
bacalhau docker run \
-v QmYq9ipYf3vsj7iLv5C67BXZcpLHxZbvFAJbtj7aKN5qii:/helloworld.pl \
--wait \
--id-only \
swipl \
 -- swipl -q -s helloworld.pl -g hello_world

then we will mount the script to the container using the -v flag

-v < CID >:/< name-of-the-script >

Swipl flag
-q running in quiet mode
-s load file as a script in this case we want to run the script helloworld.pl
-g is the name of the function you want to execute in this case its hello_world


In [ ]:
%env JOB_ID={job_id}

env: JOB_ID=ddd22db1-61c7-4302-bbe2-f38f8ffa2f44


In [ ]:
%%bash
bacalhau list --id-filter ${JOB_ID} --wide

 CREATED            ID                                    JOB                                                    STATE      VERIFIED  PUBLISHED                                            
 22-11-12-08:00:51  ddd22db1-61c7-4302-bbe2-f38f8ffa2f44  Docker swipl swipl -q -s helloworld.pl -g hello_world  Completed            /ipfs/QmYnaUZLWmbRTJzpx6kgxoAVT3ZAmhqWY6qWZm33v8PjNm 


Where it says "Completed", that means the job is done, and we can get the results.

To find out more information about your job, run the following command:


In [ ]:
%%bash
bacalhau describe ${JOB_ID}

In [ ]:
%%bash
rm -rf results && mkdir -p results
bacalhau get $JOB_ID --output-dir results

Fetching results of job 'ddd22db1-61c7-4302-bbe2-f38f8ffa2f44'...
Results for job 'ddd22db1-61c7-4302-bbe2-f38f8ffa2f44' have been written to...
results


2022/11/12 08:01:59 failed to sufficiently increase receive buffer size (was: 208 kiB, wanted: 2048 kiB, got: 416 kiB). See https://github.com/lucas-clemente/quic-go/wiki/UDP-Receive-Buffer-Size for details.


Viewing the outputs

In [ ]:
%%bash
cat results/combined_results/stdout

Hello World
